In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split



/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
cols = ['Age', 
        'Gender',
        'Country',
        'self_employed',
        'work_interfere', 
        'no_employees',
        'remote_work',
        'tech_company',
        'benefits',
        'care_options',
        'wellness_program',
        'seek_help',
        'leave',
        'coworkers',
        'mental_vs_physical', 
        'mental_health_consequence']

unprocessed_data = pd.read_csv('survey.csv', usecols=cols)

In [5]:
unprocessed_data.head()

,Age,Gender,Country,self_employed,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,leave,mental_health_consequence,coworkers,mental_vs_physical
0,37,Female,United States,NaN,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Somewhat easy,No,Some of them,Yes
1,44,M,United States,NaN,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Maybe,No,Don't know
2,32,Male,Canada,NaN,Rarely,6-25,No,Yes,No,No,No,No,Somewhat difficult,No,Yes,No
3,31,Male,United Kingdom,NaN,Often,26-100,No,Yes,No,Yes,No,No,Somewhat difficult,Yes,Some of them,No
4,31,Male,United States,NaN,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,No,Some of them,Don't know


In [6]:
len(unprocessed_data)

1259

In [8]:
unprocessed_mat = unprocessed_data.values.tolist()
age = []
male_female = []
trans = []

in_states = []

self_employed_yes_no = []
self_employed_na = []

work_interfere_often = []
work_interfere_rarely = []
work_interfere_na = []
work_interfere_sometimes = []
work_interfere_never = []

no_employees = []

# remote work features
remote_work_yes_no = []
# tech company features
tech_company_yes_no = []
# benefits features
benefits_yes_no = []
benefits_dont_know = []
# care options features
care_options_yes_no = []
care_options_not_sure = []
# wellness program features
wellness_yes_no = []
wellness_dont_know = []
# seek help features
seek_help_yes_no = []
seek_help_dont_know = []
# leave features
leave_somewhat_easy = []
leave_somewhat_difficult = []
leave_very_easy = []
leave_very_difficult = []
leave_dont_know = []
# coworker features
coworkers_yes_no = []
coworkers_some_of_them = []
# mental vs. physical features
mental_vs_physical_yes_no = []
mental_vs_physical_dont_know = []

# Output
mental_health_consequence_yes_no = []
mental_health_consequence_maybe = []

for row in range(0, len(unprocessed_mat)):
    # age
    if unprocessed_mat[row][0] < 0 or unprocessed_mat[row][0] > 80:
        age.append(0)
    else:
        age.append(unprocessed_mat[row][0])
    
    # Gender 
    male_labels = ['m','male','male-ish', 'cis man', 'cis male', 'mal', 'male (cis)', 'man', 'malr', 'man', 'msle']
    female_labels = ['f', 'female', 'woman', 'cis female', 'cis-female/femme', 'femail', 'femake']
    if unprocessed_mat[row][1].lower() in male_labels:
        male_female.append(0)
        trans.append(0)
    elif unprocessed_mat[row][1].lower() in female_labels:
        male_female.append(1)
        trans.append(0)
    else:
        male_female.append(0)
        trans.append(1)
    
    # Location
    if unprocessed_mat[row][2] == 'United States':
        in_states.append(1)
    else:
        in_states.append(0)
    
    # self_employed
    if unprocessed_mat[row][3] == 'No':
        self_employed_yes_no.append(0)
        self_employed_na.append(0)
    elif unprocessed_mat[row][3] == 'Yes':
        self_employed_yes_no.append(1)
        self_employed_na.append(0)
    else:
        self_employed_yes_no.append(0)
        self_employed_na.append(1)
    
    # work interfere
    if unprocessed_mat[row][4] == 'Often':
        work_interfere_often.append(1)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Rarely':
        work_interfere_often.append(0)
        work_interfere_rarely.append(1)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Sometimes':
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(1)
        work_interfere_never.append(0)
        work_interfere_na.append(0)
    elif unprocessed_mat[row][4] == 'Never':
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(1)
        work_interfere_na.append(0)
    else:
        work_interfere_often.append(0)
        work_interfere_rarely.append(0)
        work_interfere_sometimes.append(0)
        work_interfere_never.append(0)
        work_interfere_na.append(1)
        
    # number of employees
    if unprocessed_mat[row][5] == 'More than 1000':
        no_employees.append(1000)
    else:
        arr = unprocessed_mat[row][5].split('-')
        no_employees.append(arr[0])
        
    remote_work = unprocessed_mat[row][6].lower()
    if remote_work == "yes":
        remote_work_yes_no.append(1)
    elif remote_work == "no":
        remote_work_yes_no.append(0)
        
    tech_company = unprocessed_mat[row][7].lower()
    if tech_company == "yes":
        tech_company_yes_no.append(1)
    elif tech_company == "no":
        tech_company_yes_no.append(0)
    
    benefits = unprocessed_mat[row][8].lower()
    if benefits == "yes":
        benefits_yes_no.append(1)
        benefits_dont_know.append(0)
    elif benefits == "no":
        benefits_yes_no.append(0)
        benefits_dont_know.append(0)
    elif benefits == "don't know":
        benefits_yes_no.append(0)
        benefits_dont_know.append(1)
        
    care_options = unprocessed_mat[row][9].lower()
    if care_options == "yes":
        care_options_yes_no.append(1)
        care_options_not_sure.append(0)
    elif care_options == "no":
        care_options_yes_no.append(0)
        care_options_not_sure.append(0)
    elif care_options == "not sure":
        care_options_yes_no.append(0)
        care_options_not_sure.append(1)
    
    wellness = unprocessed_mat[row][10].lower()
    if wellness == "yes":
        wellness_yes_no.append(1)
        wellness_dont_know.append(0)
    elif wellness == "no":
        wellness_yes_no.append(0)
        wellness_dont_know.append(0)
    elif wellness == "don't know":
        wellness_yes_no.append(0)
        wellness_dont_know.append(1)
        
    seek_help = unprocessed_mat[row][11].lower()
    if seek_help == "yes":
        seek_help_yes_no.append(1)
        seek_help_dont_know.append(0)
    elif seek_help == "no":
        seek_help_yes_no.append(0)
        seek_help_dont_know.append(0)
    elif seek_help == "don't know":
        seek_help_yes_no.append(0)
        seek_help_dont_know.append(1)
        
    leave = unprocessed_mat[row][12].lower()
    if leave == "somewhat easy":
        leave_somewhat_easy.append(1)
        leave_somewhat_difficult.append(0)
        leave_very_easy.append(0)
        leave_very_difficult.append(0)
        leave_dont_know.append(0)
    elif leave == "somewhat difficult":
        leave_somewhat_easy.append(0)
        leave_somewhat_difficult.append(1)
        leave_very_easy.append(0)
        leave_very_difficult.append(0)
        leave_dont_know.append(0)
    elif leave == "very easy":
        leave_somewhat_easy.append(0)
        leave_somewhat_difficult.append(0)
        leave_very_easy.append(1)
        leave_very_difficult.append(0)
        leave_dont_know.append(0)
    elif leave == "very difficult":
        leave_somewhat_easy.append(0)
        leave_somewhat_difficult.append(0)
        leave_very_easy.append(0)
        leave_very_difficult.append(1)
        leave_dont_know.append(0)
    elif leave == "don't know":
        leave_somewhat_easy.append(0)
        leave_somewhat_difficult.append(0)
        leave_very_easy.append(0)
        leave_very_difficult.append(0)
        leave_dont_know.append(1)
        
    coworkers = unprocessed_mat[row][14].lower()
    if coworkers == "yes":
        coworkers_yes_no.append(1)
        coworkers_some_of_them.append(0)
    elif coworkers == "no":
        coworkers_yes_no.append(0)
        coworkers_some_of_them.append(0)
    elif coworkers == "some of them":
        coworkers_yes_no.append(0)
        coworkers_some_of_them.append(1)
        
    mental_vs_physical = unprocessed_mat[row][15].lower()
    if mental_vs_physical == "yes":
        mental_vs_physical_yes_no.append(1)
        mental_vs_physical_dont_know.append(0)
    elif mental_vs_physical == "no":
        mental_vs_physical_yes_no.append(0)
        mental_vs_physical_dont_know.append(0)
    elif mental_vs_physical == "don't know":
        mental_vs_physical_yes_no.append(0)
        mental_vs_physical_dont_know.append(1)
        
    mental_health_consequence = unprocessed_mat[row][13].lower()
    if mental_health_consequence == "yes":
        mental_health_consequence_yes_no.append(1)
        mental_health_consequence_maybe.append(0)
    elif mental_health_consequence == "no":
        mental_health_consequence_yes_no.append(0)
        mental_health_consequence_maybe.append(0)
    elif mental_health_consequence == "maybe":
        mental_health_consequence_yes_no.append(0)
        mental_health_consequence_maybe.append(1)
    
        